### Ejercicio 3: Introducción a la optimización

La empresa Sunrise Breakfast Company debe producir 2000 libras al día de
copos de cereales para satisfacer la demanda de su producto Sugar Sweet
Cereal. El coste por cada libra de los ingredientes de que se compone el
producto son: Ingrediente A 4 \$/libra; B 3 \$/libra; C 2\$/libra. Los
requisitos del gobierno establecen que la mezcla deba contener al menos
el 10% del ingrediente A y el 20% del ingrediente B. Se sabe también que
el uso de más de 800 libras por ton (1 ton=2000 libras) produce un saber
inaceptable. Determinar el mínimo coste de la mezcla que satisfaga la
demana de Sugar Sweet Cereal.

#### Importamos Pyomo

In [25]:
from pyomo.environ import *

#### Creamos un problema concreto, es decir, un objeto de la clase problema que ya contenga parámetros

In [26]:
model = ConcreteModel()

#### Definimos las variables como no negativas

In [27]:
model.A = Var(domain=NonNegativeReals)
model.B = Var(domain=NonNegativeReals)
model.C = Var(domain=NonNegativeReals)


#### Definimos la función objetivo

In [28]:
model.profit = Objective(expr = 4*model.A+3*model.B+2*model.C,
                         sense=minimize)

#### Definimos la restricciones

In [29]:
model.CantA = Constraint(expr = model.A-0.1*(model.A+model.B+model.C) >= 0)
model.CantB = Constraint(expr = model.B-0.2*(model.A+model.B+model.C) >= 0)
model.CantC = Constraint(expr = model.C <= 800)
model.Prod  = Constraint(expr = model.A+model.B+model.C == 2000)


In [30]:
results = SolverFactory('glpk').solve(model)
model.pprint()
results.write()   

3 Var Declarations
    A : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 : 200.0 :  None : False : False : NonNegativeReals
    B : Size=1, Index=None
        Key  : Lower : Value  : Upper : Fixed : Stale : Domain
        None :     0 : 1000.0 :  None : False : False : NonNegativeReals
    C : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 : 800.0 :  None : False : False : NonNegativeReals

1 Objective Declarations
    profit : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : 4*A + 3*B + 2*C

4 Constraint Declarations
    CantA : Size=1, Index=None, Active=True
        Key  : Lower : Body                : Upper : Active
        None :   0.0 : A - 0.1*(A + B + C) :  +Inf :   True
    CantB : Size=1, Index=None, Active=True
        Key  : Lower : Body                : Upper : Active
        None :   0.0 : B - 0.2*(A